# Descripción del proyecto
Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

* Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
* Crear un modelo para predecir el volumen de reservas en pozos nuevos.
* Elegir los pozos petrolíferos que tienen los valores estimados más altos.
* Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

---

# Condiciones:
* Solo se debe usar la regresión lineal para el entrenamiento del modelo.
* Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
* El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
* Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
* Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.

Los datos son sintéticos: los detalles del contrato y las características del pozo no se publican.

---

Descripción de datos
Los datos de exploración geológica de las tres regiones se almacenan en archivos:

* /datasets/geo_data_0.csv. Descarga el conjunto de datos
* /datasets/geo_data_1.csv. Descarga el conjunto de datos
* /datasets/geo_data_2.csv. Descarga el conjunto de datos
* id — identificador único de pozo de petróleo
* f0, f1, f2 — tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)
* product — volumen de reservas en el pozo de petróleo (miles de barriles).

---

# Instrucciones del proyecto
1. Descarga y prepara los datos. Explica el procedimiento.
2. Entrena y prueba el modelo para cada región en geo_data_0.csv:
    1. Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25
    2. Entrena el modelo y haz predicciones para el conjunto de validación.
    3. Guarda las predicciones y las respuestas correctas para el conjunto de validación.
    4. Muestra el volumen medio de reservas predicho y RMSE del modelo.
    5. Analiza los resultados.
    6. Coloca todos los pasos previos en funciones, realiza y ejecuta los pasos 2.1-2.5 para los archivos 'geo_data_1.csv' y 'geo_data_2.csv'.
3. Prepárate para el cálculo de ganancias:
    1. Almacena todos los valores necesarios para los cálculos en variables separadas.
    2. Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). Compara esta cantidad con la cantidad media de reservas en cada región.
    3. Presenta conclusiones sobre cómo preparar el paso para calcular el beneficio.
4. Escribe una función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados y modela las predicciones:
    1. Elige los 200 pozos con los valores de predicción más altos de cada una de las 3 regiones (es decir, archivos 'csv').
    2. Resume el volumen objetivo de reservas según dichas predicciones. Almacena las predicciones para los 200 pozos para cada una de las 3 regiones.
    3. Calcula la ganancia potencial de los 200 pozos principales por región. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección.
5. Calcula riesgos y ganancias para cada región:
    1. Utilizando las predicciones que almacenaste en el paso 4.2, emplea la técnica del bootstrapping con 1000 muestras para hallar la distribución de los beneficios.
    2. Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje.
    3. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección. ¿Coincide tu elección con la elección anterior en el punto 4.3?

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

# Preparación de los datos

In [2]:
url_0 = 'https://raw.githubusercontent.com/Davichobacter/data_science_tt/refs/heads/main/Sprint_11/data/geo_data_0.csv'
url_1 = 'https://raw.githubusercontent.com/Davichobacter/data_science_tt/refs/heads/main/Sprint_11/data/geo_data_1.csv'
url_2 = 'https://raw.githubusercontent.com/Davichobacter/data_science_tt/refs/heads/main/Sprint_11/data/geo_data_2.csv'

In [3]:
df_0 = pd.read_csv(url_0)
df_1 = pd.read_csv(url_1)
df_2 = pd.read_csv(url_2)

In [4]:
def explorar_dataset(df):
    print('---' * 10, '\n', f'Información del dataframe')
    print(df.info())
    print('---' * 10, '\n', f'Dimensiones del dataframe')
    print(df.shape)
    print('---' * 10, '\n', f'Primeras filas del dataframe')
    print(df.head(15))
    print('---' * 10, '\n', f'Descripción del dataframe')
    print(df.describe())
    print('---' * 10, '\n', f'Valores nulos del dataframe')
    print(df.isnull().sum())
    print('---' * 10, '\n', f'Valores duplicados del dataframe')
    print(df.duplicated().sum())
    print('---' * 10)

## Dataframe 0

In [5]:
explorar_dataset(df_0)

------------------------------ 
 Información del dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
------------------------------ 
 Dimensiones del dataframe
(100000, 5)
------------------------------ 
 Primeras filas del dataframe
       id        f0        f1        f2     product
0   txEyH  0.705745 -0.497823  1.221170  105.280062
1   2acmU  1.334711 -0.340164  4.365080   73.037750
2   409Wp  1.022732  0.151990  1.419926   85.265647
3   iJLyR -0.032172  0.139033  2.978566  168.620776
4   Xdl7t  1.988431  0.155413  4.751769  154.036647
5   wX4Hy  0.969570  0.489775 -0.735383   64.741541
6   t

## Dataframe 1

In [6]:
explorar_dataset(df_1)

------------------------------ 
 Información del dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
------------------------------ 
 Dimensiones del dataframe
(100000, 5)
------------------------------ 
 Primeras filas del dataframe
       id         f0         f1        f2     product
0   kBEdx -15.001348  -8.276000 -0.005876    3.179103
1   62mP7  14.272088  -3.475083  0.999183   26.953261
2   vyE1P   6.263187  -5.948386  5.001160  134.766305
3   KcrkZ -13.081196 -11.506057  4.999415  137.945408
4   AHL4O  12.702195  -8.147433  5.004363  134.766305
5   HHckp  -3.327590  -2.205276  3.003647   8

## Dataframe 2

In [7]:
explorar_dataset(df_2)

------------------------------ 
 Información del dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
------------------------------ 
 Dimensiones del dataframe
(100000, 5)
------------------------------ 
 Primeras filas del dataframe
       id        f0        f1        f2     product
0   fwXo0 -1.146987  0.963328 -0.828965   27.758673
1   WJtFt  0.262778  0.269839 -2.530187   56.069697
2   ovLUW  0.194587  0.289035 -5.586433   62.871910
3   q6cA6  2.236060 -0.553760  0.930038  114.572842
4   WPMUX -0.515993  1.716266  5.899011  149.600746
5   LzZXx -0.758092  0.710691  2.585887   90.222465
6   W

# Entrenar el modelo para cada Región

In [14]:
def entrenar_modelo(df):
    """
    Función que divide los datos en un conjunto de entrenamiento y un conjunto
    de validación en una proporción de 75:25. Entrena el modelo y hace predicciones
    para el conjunto de validación. Guarda las predicciones y las respuestas correctas
    para el conjunto de validación. Muestra el volumen medio de reservas predicho
    y RMSE del modelo.

    Parámetros
        df: DataFrame con los datos de entrenamiento

    Retorna
        predicted_valid: predicciones del modelo
        target_valid: respuestas correctas del modelo
        mean_reserves: volumen medio de reservas predicho
        rmse: RMSE del modelo
    """

    features_train, features_valid, target_train, target_valid = train_test_split(df, df['product'], test_size=0.25)

    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)

    mean_reserves = round(predicted_valid.mean(), 3)
    rmse = root_mean_squared_error(target_valid, predicted_valid)

    print(f'Volumen medio de reservas: {mean_reserves}')
    print(f'RMSE: {round(rmse, 3)}')

    return predicted_valid, target_valid, mean_reserves, rmse

In [15]:
df_0_final = df_0.drop(['id'], axis=1)
df_1_final = df_1.drop(['id'], axis=1)
df_2_final = df_2.drop(['id'], axis=1)

## Entrenamiento del modelo para la Región 0

In [16]:
region_0_predicted, region_0_target, region_0_mean, region_0_rmse = entrenar_modelo(df_0_final)

Volumen medio de reservas: 92.98
RMSE: 0.0


## Entrenamiento del modelo para la Región 1

In [11]:
region_1_predicted, region_1_target, region_1_mean, region_1_rmse = entrenar_modelo(df_1_final, 12345)

Volumen medio de reservas: 68.723
RMSE: 0.0


## Entrenamiento del modelo para la Región 2

In [12]:
region_2_predicted, region_2_target, region_2_mean, region_2_rmse = entrenar_modelo(df_2_final, 12345)

Volumen medio de reservas: 94.884
RMSE: 0.0


# Preparación para cálculo de ganancias

# Task
Prepare for profit calculation by defining financial constants (budget for well development, revenue per unit of product, number of top wells to select, and risk threshold), calculating the breakeven product volume per well, and comparing this breakeven volume with the mean predicted reserves for each region. Based on these comparisons, provide conclusions on which regions are likely to be profitable and how to proceed.

## Define Financial Constants

### Subtask:
Store all the necessary financial values such as budget for well development, revenue per unit of product, and the number of top wells to select in separate variables. Also, define the risk threshold.


**Reasoning**:
The subtask requires defining several financial constants. This will be done by creating variables for the budget, revenue per unit, number of top wells, and risk threshold, and assigning the values specified in the instructions.



In [27]:
budget_for_wells = 100_000_000
revenue_per_unit = 4500
num_top_wells = 200
risk_threshold = 0.025

**Reasoning**:
The previous step defined the financial constants. Now, according to instruction 3.2, I need to calculate the breakeven product volume per well and compare it with the mean predicted reserves for each region.



In [28]:
cost_per_well = budget_for_wells / num_top_wells
breakeven_product_volume = cost_per_well / revenue_per_unit

print(f"Costo por pozo: ${cost_per_well:,.2f}")
print(f"Volumen mínimo de reservas para evitar pérdidas por pozo: {breakeven_product_volume:.3f} mil barriles\n")

print(f"Volumen medio de reservas predicho para la Región 0: {region_0_mean:.3f} mil barriles")
print(f"Volumen medio de reservas predicho para la Región 1: {region_1_mean:.3f} mil barriles")
print(f"Volumen medio de reservas predicho para la Región 2: {region_2_mean:.3f} mil barriles")

Costo por pozo: $500,000.00
Volumen mínimo de reservas para evitar pérdidas por pozo: 111.111 mil barriles

Volumen medio de reservas predicho para la Región 0: 92.980 mil barriles
Volumen medio de reservas predicho para la Región 1: 68.723 mil barriles
Volumen medio de reservas predicho para la Región 2: 94.884 mil barriles


## Conclusiones sobre la preparación para el cálculo del beneficio

Con un presupuesto de 100 millones de dólares para desarrollar 200 pozos, el costo promedio por pozo es de 500,000 dólares. Dado que cada unidad de producto genera 4,500 dólares, cada pozo necesita producir al menos **111.111 mil barriles** para cubrir los costos y evitar pérdidas.

Al comparar este volumen mínimo de reservas con los volúmenes medios de reservas predichos para cada región, observamos lo siguiente:

*   **Región 0:** El volumen medio de reservas predicho es de **92.980 mil barriles**.
*   **Región 1:** El volumen medio de reservas predicho es de **68.723 mil barriles**.
*   **Región 2:** El volumen medio de reservas predicho es de **94.884 mil barriles**.

En todas las regiones, el volumen medio de reservas predicho es **inferior** al volumen mínimo requerido de 111.111 mil barriles para alcanzar el punto de equilibrio. Esto indica que, en promedio, desarrollar pozos en cualquiera de estas regiones tal como está, **conllevaría pérdidas si se selecciona aleatoriamente**. Por lo tanto, la selección de los 200 mejores pozos con los valores de predicción más altos será crucial para intentar superar este umbral y asegurar la rentabilidad. Esta comparación inicial subraya la importancia de la fase de selección de pozos y la necesidad de un análisis de riesgo y beneficios más profundo con bootstrapping para identificar la región más prometedora.

## Select Top 200 Wells by Prediction

### Subtask:
Select the 200 wells with the highest predicted product volume for each of the three regions.


**Reasoning**:
To select the top 200 predicted wells, I will convert the numpy arrays of predictions for each region into pandas Series, sort them in descending order, and then take the top 200 values.



In [29]:
top_200_region_0 = pd.Series(region_0_predicted).sort_values(ascending=False).head(num_top_wells)
top_200_region_1 = pd.Series(region_1_predicted).sort_values(ascending=False).head(num_top_wells)
top_200_region_2 = pd.Series(region_2_predicted).sort_values(ascending=False).head(num_top_wells)

print(f"Top 200 predicted product volumes for Region 0 (first 5):\n{top_200_region_0.head()}")
print(f"\nTop 200 predicted product volumes for Region 1 (first 5):\n{top_200_region_1.head()}")
print(f"\nTop 200 predicted product volumes for Region 2 (first 5):\n{top_200_region_2.head()}")

Top 200 predicted product volumes for Region 0 (first 5):
9665    185.362690
6918    185.337352
8815    185.330529
8141    185.317346
8624    185.304563
dtype: float64

Top 200 predicted product volumes for Region 1 (first 5):
14432    137.945408
17711    137.945408
14375    137.945408
14440    137.945408
17729    137.945408
dtype: float64

Top 200 predicted product volumes for Region 2 (first 5):
11365    190.010982
20144    189.981304
14563    189.968677
18172    189.966835
7564     189.948611
dtype: float64


## Summarize Predicted Volumes for Top 200 Wells

### Subtask:
Calculate the total predicted volume of reserves for the top 200 wells selected in each of the three regions. Store these summarized volumes in separate variables.

**Reasoning**:
The previous step outlined the need to calculate the total predicted volume of reserves for the top 200 wells. Now, I will implement this by summing the `top_200_region_X` Series for each region and storing the results.



In [30]:
total_predicted_volume_0 = top_200_region_0.sum()
total_predicted_volume_1 = top_200_region_1.sum()
total_predicted_volume_2 = top_200_region_2.sum()

print(f"Total predicted volume for top 200 wells in Region 0: {total_predicted_volume_0:.3f} mil barriles")
print(f"Total predicted volume for top 200 wells in Region 1: {total_predicted_volume_1:.3f} mil barriles")
print(f"Total predicted volume for top 200 wells in Region 2: {total_predicted_volume_2:.3f} mil barriles")

Total predicted volume for top 200 wells in Region 0: 36677.439 mil barriles
Total predicted volume for top 200 wells in Region 1: 27589.082 mil barriles
Total predicted volume for top 200 wells in Region 2: 37650.363 mil barriles


**Reasoning**:
The previous steps selected the top 200 wells and calculated their total predicted volumes. Now, I need to calculate the potential profit for each region based on these selected wells, which is instruction 4.3 of the main task. To do this efficiently and in preparation for the bootstrapping in step 5, I will first define a function to calculate the profit for a given set of wells. This function will take the `target` (actual product volumes) and `predictions` (predicted product volumes) as input, select the top `num_top_wells` based on predictions, and then calculate the total profit using the defined financial constants.



In [31]:
def calculate_profit(target, predictions, num_top_wells, budget_for_wells, revenue_per_unit):
    # Sort predictions in descending order and get the indices of the top N wells
    predictions_series = pd.Series(predictions)
    sorted_predictions = predictions_series.sort_values(ascending=False)
    top_wells_indices = sorted_predictions.head(num_top_wells).index

    # Get the actual product volumes for the selected top N wells
    selected_actual_volumes = target.iloc[top_wells_indices]

    # Sum the actual product volumes
    total_product_volume = selected_actual_volumes.sum()

    # Calculate total revenue
    total_revenue = total_product_volume * revenue_per_unit

    # Calculate profit
    profit = total_revenue - budget_for_wells

    return profit

print("Function 'calculate_profit' defined.")

Function 'calculate_profit' defined.


**Reasoning**:
The `calculate_profit` function has been defined. Now, I will use this function to calculate the potential profit for each region, as specified in instruction 4.3 of the main task. I will pass the target values, predicted values, and financial constants to the function for each region.



In [32]:
profit_0 = calculate_profit(region_0_target, region_0_predicted, num_top_wells, budget_for_wells, revenue_per_unit)
profit_1 = calculate_profit(region_1_target, region_1_predicted, num_top_wells, budget_for_wells, revenue_per_unit)
profit_2 = calculate_profit(region_2_target, region_2_predicted, num_top_wells, budget_for_wells, revenue_per_unit)

print(f"Ganancia potencial para la Región 0: ${profit_0:,.2f}")
print(f"Ganancia potencial para la Región 1: ${profit_1:,.2f}")
print(f"Ganancia potencial para la Región 2: ${profit_2:,.2f}")

Ganancia potencial para la Región 0: $65,048,474.95
Ganancia potencial para la Región 1: $24,150,866.97
Ganancia potencial para la Región 2: $69,426,631.59


## Conclusiones sobre la Ganancia Potencial por Región

Después de seleccionar los 200 pozos con los valores de predicción más altos y calcular la ganancia potencial para cada región, los resultados son los siguientes:

*   **Región 0:** Ganancia potencial de **$65,048,474.95**
*   **Región 1:** Ganancia potencial de **$24,150,866.97**
*   **Región 2:** Ganancia potencial de **$69,426,631.59**

### Propuesta de Región para el Desarrollo

Basándonos únicamente en la ganancia potencial calculada para los 200 pozos más prometedores, la **Región 2** presenta el beneficio más alto, con **$69,426,631.59**. La Región 0 le sigue de cerca con un beneficio de $65,048,474.95, mientras que la Región 1 muestra el beneficio más bajo.

Por lo tanto, la **Región 2** es la candidata más fuerte para el desarrollo de pozos petrolíferos en este punto, ya que ofrece la mayor ganancia esperada bajo el presupuesto y la selección de pozos definidos. Sin embargo, es crucial recordar que este análisis aún no ha incorporado la evaluación de riesgos, que es el siguiente paso y podría influir significativamente en la decisión final.

## Conclusiones sobre la Ganancia Potencial por Región

Después de seleccionar los 200 pozos con los valores de predicción más altos y calcular la ganancia potencial para cada región, los resultados son los siguientes:

*   **Región 0:** Ganancia potencial de **$65,048,474.95**
*   **Región 1:** Ganancia potencial de **$24,150,866.97**
*   **Región 2:** Ganancia potencial de **$69,426,631.59**

### Propuesta de Región para el Desarrollo

Basándonos únicamente en la ganancia potencial calculada para los 200 pozos más prometedores, la **Región 2** presenta el beneficio más alto, con **$69,426,631.59**. La Región 0 le sigue de cerca con un beneficio de $65,048,474.95, mientras que la Región 1 muestra el beneficio más bajo.

Por lo tanto, la **Región 2** es la candidata más fuerte para el desarrollo de pozos petrolíferos en este punto, ya que ofrece la mayor ganancia esperada bajo el presupuesto y la selección de pozos definidos. Sin embargo, es crucial recordar que este análisis aún no ha incorporado la evaluación de riesgos, que es el siguiente paso y podría influir significativamente en la decisión final.

## Conclusiones sobre la Ganancia Potencial por Región

Después de seleccionar los 200 pozos con los valores de predicción más altos y calcular la ganancia potencial para cada región, los resultados son los siguientes:

*   **Región 0:** Ganancia potencial de **$65,048,474.95**
*   **Región 1:** Ganancia potencial de **$24,150,866.97**
*   **Región 2:** Ganancia potencial de **$69,426,631.59**

### Propuesta de Región para el Desarrollo

Basándonos únicamente en la ganancia potencial calculada para los 200 pozos más prometedores, la **Región 2** presenta el beneficio más alto, con **$69,426,631.59**. La Región 0 le sigue de cerca con un beneficio de $65,048,474.95, mientras que la Región 1 muestra el beneficio más bajo.

Por lo tanto, la **Región 2** es la candidata más fuerte para el desarrollo de pozos petrolíferos en este punto, ya que ofrece la mayor ganancia esperada bajo el presupuesto y la selección de pozos definidos. Sin embargo, es crucial recordar que este análisis aún no ha incorporado la evaluación de riesgos, que es el siguiente paso y podría influir significativamente en la decisión final.


## Calculate risks and profits for each region using Bootstrapping

### Subtask:
Calculate the average profit, 95% confidence interval, and risk of losses for each region using bootstrapping with 1000 samples.


**Reasoning**:
I need to implement the bootstrapping process as described in the instructions. This involves initializing lists to store profits, iterating 1000 times for each region, randomly sampling data, calculating profit using the previously defined function, and then computing the mean profit, 95% confidence interval, and risk of loss.



In [33]:
profits_0 = []
profits_1 = []
profits_2 = []

# Number of bootstrapping samples
N_SAMPLES = 1000

# Convert predictions to Series for easier indexing with target Series
predictions_0_series = pd.Series(region_0_predicted)
predictions_1_series = pd.Series(region_1_predicted)
predictions_2_series = pd.Series(region_2_predicted)

# Ensure target is a Series with a reset index if it's not already contiguously indexed
target_0_reset = region_0_target.reset_index(drop=True)
target_1_reset = region_1_target.reset_index(drop=True)
target_2_reset = region_2_target.reset_index(drop=True)


print("Calculating profits and risks for Region 0...")
for i in range(N_SAMPLES):
    # Randomly sample indices from the validation set (size of validation set is 25000)
    # The instruction says 500 points for exploration, but for bootstrapping,
    # we usually sample from the existing validation set size for each iteration.
    # Given the previous split of 75:25, the validation set size is 25000.
    # The project description mentions 'a study of 500 points with the selection of the best 200 points'.
    # This implies that from a larger set of potential points (like the validation set),
    # we explore 500 and then pick the best 200. For bootstrapping, we should sample 500 from the validation set.
    # However, the calculate_profit function selects top_wells (200) from the 'predictions' input.
    # So, for the bootstrap sample, we need to ensure we have enough points to select 200 from.
    # Let's sample 500 indices as per project description's exploration study size.

    # Adjusting to sample from the full validation set (25000) for bootstrapping
    # and then applying the top 200 selection within the calculate_profit function

    # If we need to simulate the 'study of 500 points' then we should sample 500 from validation set.
    # Let's assume for bootstrapping, we re-sample from the entire validation set.
    # The previous instruction 3.b says 'randomly sample 500 indices (with replacement) from the `target` and `predicted` series'
    # This 500-point sampling should be done from the full validation set (25000 entries).
    # Then, the 'calculate_profit' function will select the best 200 from these 500 sampled points.

    # Let's re-evaluate the sampling logic based on the project description:
    # "Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio."
    # This implies that for each iteration of bootstrapping, we would first select 500 random points,
    # and then from these 500 points, we would select the top 200 for profit calculation.
    # This requires modifying the calculate_profit function or its call.

    # Let's assume the current `calculate_profit` function is designed to work with the full set of predictions/targets passed to it,
    # and it selects the top `num_top_wells` from that *entire* set.
    # For bootstrapping, we need to resample from our validation set.

    # The instruction 3.b is key: 'randomly sample 500 indices (with replacement) from the target and predicted series'.
    # This means each bootstrap sample will consist of 500 wells.
    # Then, within `calculate_profit`, we'll select the top 200 from these 500.

    sample_indices = np.random.choice(target_0_reset.index, size=500, replace=True)
    target_sample = target_0_reset.iloc[sample_indices]
    predictions_sample = predictions_0_series.iloc[sample_indices]
    profit = calculate_profit(target_sample, predictions_sample, num_top_wells, budget_for_wells, revenue_per_unit)
    profits_0.append(profit)

print("Calculating profits and risks for Region 1...")
for i in range(N_SAMPLES):
    sample_indices = np.random.choice(target_1_reset.index, size=500, replace=True)
    target_sample = target_1_reset.iloc[sample_indices]
    predictions_sample = predictions_1_series.iloc[sample_indices]
    profit = calculate_profit(target_sample, predictions_sample, num_top_wells, budget_for_wells, revenue_per_unit)
    profits_1.append(profit)

print("Calculating profits and risks for Region 2...")
for i in range(N_SAMPLES):
    sample_indices = np.random.choice(target_2_reset.index, size=500, replace=True)
    target_sample = target_2_reset.iloc[sample_indices]
    predictions_sample = predictions_2_series.iloc[sample_indices]
    profit = calculate_profit(target_sample, predictions_sample, num_top_wells, budget_for_wells, revenue_per_unit)
    profits_2.append(profit)

# Convert lists to Series for calculations
profits_0 = pd.Series(profits_0)
profits_1 = pd.Series(profits_1)
profits_2 = pd.Series(profits_2)

# Calculate statistics for Region 0
mean_profit_0 = profits_0.mean()
confidence_interval_0 = np.percentile(profits_0, [2.5, 97.5])
risk_of_loss_0 = (profits_0 < 0).mean() * 100

# Calculate statistics for Region 1
mean_profit_1 = profits_1.mean()
confidence_interval_1 = np.percentile(profits_1, [2.5, 97.5])
risk_of_loss_1 = (profits_1 < 0).mean() * 100

# Calculate statistics for Region 2
mean_profit_2 = profits_2.mean()
confidence_interval_2 = np.percentile(profits_2, [2.5, 97.5])
risk_of_loss_2 = (profits_2 < 0).mean() * 100

# Print results
print("\n--- Region 0 Results ---")
print(f"Beneficio promedio: ${mean_profit_0:,.2f}")
print(f"Intervalo de confianza del 95%: [${confidence_interval_0[0]:,.2f}, ${confidence_interval_0[1]:,.2f}]")
print(f"Riesgo de pérdidas: {risk_of_loss_0:.2f}%")

print("\n--- Region 1 Results ---")
print(f"Beneficio promedio: ${mean_profit_1:,.2f}")
print(f"Intervalo de confianza del 95%: [${confidence_interval_1[0]:,.2f}, ${confidence_interval_1[1]:,.2f}]")
print(f"Riesgo de pérdidas: {risk_of_loss_1:.2f}%")

print("\n--- Region 2 Results ---")
print(f"Beneficio promedio: ${mean_profit_2:,.2f}")
print(f"Intervalo de confianza del 95%: [${confidence_interval_2[0]:,.2f}, ${confidence_interval_2[1]:,.2f}]")
print(f"Riesgo de pérdidas: {risk_of_loss_2:.2f}%")

Calculating profits and risks for Region 0...


IndexError: positional indexers are out-of-bounds

**Reasoning**:
The previous code failed with an `IndexError` because the `calculate_profit` function used `.iloc` with indices that were labels from the original DataFrame, not positional integers, after resampling. To fix this, I will modify the `calculate_profit` function to use `.loc` instead of `.iloc` to correctly select the actual product volumes based on the original indices of the top predicted wells.



In [34]:
def calculate_profit(target, predictions, num_top_wells, budget_for_wells, revenue_per_unit):
    # Sort predictions in descending order and get the indices of the top N wells
    # predictions is already a pandas Series at this point (from the bootstrap sample)
    sorted_predictions = predictions.sort_values(ascending=False)
    top_wells_indices = sorted_predictions.head(num_top_wells).index

    # Get the actual product volumes for the selected top N wells using .loc
    # target is also a pandas Series with aligned indices from the bootstrap sample
    selected_actual_volumes = target.loc[top_wells_indices]

    # Sum the actual product volumes
    total_product_volume = selected_actual_volumes.sum()

    # Calculate total revenue
    total_revenue = total_product_volume * revenue_per_unit

    # Calculate profit
    profit = total_revenue - budget_for_wells

    return profit

print("Function 'calculate_profit' redefined with index-aware selection.")

Function 'calculate_profit' redefined with index-aware selection.


**Reasoning**:
The `calculate_profit` function has been redefined to address the `IndexError`. Now, I will re-run the bootstrapping code with the corrected function to calculate the average profit, 95% confidence interval, and risk of losses for each region, as specified in the subtask.



In [35]:
profits_0 = []
profits_1 = []
profits_2 = []

# Number of bootstrapping samples
N_SAMPLES = 1000

# Convert predictions to Series for easier indexing with target Series
predictions_0_series = pd.Series(region_0_predicted)
predictions_1_series = pd.Series(region_1_predicted)
predictions_2_series = pd.Series(region_2_predicted)

# Ensure target is a Series with a reset index if it's not already contiguously indexed
target_0_reset = region_0_target.reset_index(drop=True)
target_1_reset = region_1_target.reset_index(drop=True)
target_2_reset = region_2_target.reset_index(drop=True)


print("Calculating profits and risks for Region 0...")
for i in range(N_SAMPLES):
    # Randomly sample indices from the validation set (size of validation set is 25000)
    # The instruction says 500 points for exploration, but for bootstrapping,
    # we usually sample from the existing validation set size for each iteration.
    # Given the previous split of 75:25, the validation set size is 25000.
    # The project description mentions 'a study of 500 points with the selection of the best 200 points'.
    # This implies that from a larger set of potential points (like the validation set),
    # we explore 500 and then pick the best 200. For bootstrapping, we should sample 500 from the validation set.
    # However, the calculate_profit function selects top_wells (200) from the 'predictions' input.
    # So, for the bootstrap sample, we need to ensure we have enough points to select 200 from.
    # Let's sample 500 indices as per project description's exploration study size.

    # Adjusting to sample from the full validation set (25000) for bootstrapping
    # and then applying the top 200 selection within the calculate_profit function

    # If we need to simulate the 'study of 500 points' then we should sample 500 from validation set.
    # Let's assume for bootstrapping, we re-sample from the entire validation set.
    # The previous instruction 3.b says 'randomly sample 500 indices (with replacement) from the `target` and `predicted` series'
    # This 500-point sampling should be done from the full validation set (25000 entries).
    # Then, the 'calculate_profit' function will select the best 200 from these 500 sampled points.

    # Let's re-evaluate the sampling logic based on the project description:
    # "Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio."
    # This implies that for each iteration of bootstrapping, we would first select 500 random points,
    # and then from these 500 points, we would select the top 200 for profit calculation.
    # This requires modifying the calculate_profit function or its call.

    # Let's assume the current `calculate_profit` function is designed to work with the full set of predictions/targets passed to it,
    # and it selects the top `num_top_wells` from that *entire* set.
    # For bootstrapping, we need to resample from our validation set.

    # The instruction 3.b is key: 'randomly sample 500 indices (with replacement) from the target and predicted series'.
    # This means each bootstrap sample will consist of 500 wells.
    # Then, within `calculate_profit`, we'll select the top 200 from these 500.

    sample_indices = np.random.choice(target_0_reset.index, size=500, replace=True)
    target_sample = target_0_reset.loc[sample_indices] # Changed .iloc to .loc
    predictions_sample = predictions_0_series.loc[sample_indices] # Changed .iloc to .loc
    profit = calculate_profit(target_sample, predictions_sample, num_top_wells, budget_for_wells, revenue_per_unit)
    profits_0.append(profit)

print("Calculating profits and risks for Region 1...")
for i in range(N_SAMPLES):
    sample_indices = np.random.choice(target_1_reset.index, size=500, replace=True)
    target_sample = target_1_reset.loc[sample_indices] # Changed .iloc to .loc
    predictions_sample = predictions_1_series.loc[sample_indices] # Changed .iloc to .loc
    profit = calculate_profit(target_sample, predictions_sample, num_top_wells, budget_for_wells, revenue_per_unit)
    profits_1.append(profit)

print("Calculating profits and risks for Region 2...")
for i in range(N_SAMPLES):
    sample_indices = np.random.choice(target_2_reset.index, size=500, replace=True)
    target_sample = target_2_reset.loc[sample_indices] # Changed .iloc to .loc
    predictions_sample = predictions_2_series.loc[sample_indices] # Changed .iloc to .loc
    profit = calculate_profit(target_sample, predictions_sample, num_top_wells, budget_for_wells, revenue_per_unit)
    profits_2.append(profit)

# Convert lists to Series for calculations
profits_0 = pd.Series(profits_0)
profits_1 = pd.Series(profits_1)
profits_2 = pd.Series(profits_2)

# Calculate statistics for Region 0
mean_profit_0 = profits_0.mean()
confidence_interval_0 = np.percentile(profits_0, [2.5, 97.5])
risk_of_loss_0 = (profits_0 < 0).mean() * 100

# Calculate statistics for Region 1
mean_profit_1 = profits_1.mean()
confidence_interval_1 = np.percentile(profits_1, [2.5, 97.5])
risk_of_loss_1 = (profits_1 < 0).mean() * 100

# Calculate statistics for Region 2
mean_profit_2 = profits_2.mean()
confidence_interval_2 = np.percentile(profits_2, [2.5, 97.5])
risk_of_loss_2 = (profits_2 < 0).mean() * 100

# Print results
print("\n--- Region 0 Results ---")
print(f"Beneficio promedio: ${mean_profit_0:,.2f}")
print(f"Intervalo de confianza del 95%: [${confidence_interval_0[0]:,.2f}, ${confidence_interval_0[1]:,.2f}]")
print(f"Riesgo de pérdidas: {risk_of_loss_0:.2f}%")

print("\n--- Region 1 Results ---")
print(f"Beneficio promedio: ${mean_profit_1:,.2f}")
print(f"Intervalo de confianza del 95%: [${confidence_interval_1[0]:,.2f}, ${confidence_interval_1[1]:,.2f}]")
print(f"Riesgo de pérdidas: {risk_of_loss_1:.2f}%")

print("\n--- Region 2 Results ---")
print(f"Beneficio promedio: ${mean_profit_2:,.2f}")
print(f"Intervalo de confianza del 95%: [${confidence_interval_2[0]:,.2f}, ${confidence_interval_2[1]:,.2f}]")
print(f"Riesgo de pérdidas: {risk_of_loss_2:.2f}%")

Calculating profits and risks for Region 0...
Calculating profits and risks for Region 1...
Calculating profits and risks for Region 2...

--- Region 0 Results ---
Beneficio promedio: $26,674,574.75
Intervalo de confianza del 95%: [$21,464,572.89, $32,462,222.97]
Riesgo de pérdidas: 0.00%

--- Region 1 Results ---
Beneficio promedio: $6,777,032.30
Intervalo de confianza del 95%: [$1,479,105.53, $12,298,424.41]
Riesgo de pérdidas: 0.50%

--- Region 2 Results ---
Beneficio promedio: $28,807,085.25
Intervalo de confianza del 95%: [$23,454,602.71, $35,023,917.94]
Riesgo de pérdidas: 0.00%


## Conclusiones y Propuesta de Región Final (Después de Bootstrapping)

Después de realizar el análisis de riesgos y beneficios utilizando la técnica de bootstrapping con 1000 muestras, los resultados para cada región son los siguientes:

*   **Región 0:**
    *   Beneficio promedio: **$26,674,574.75**
    *   Intervalo de confianza del 95%: [**$21,464,572.89**, **$32,462,222.97**]
    *   Riesgo de pérdidas: **0.00%**

*   **Región 1:**
    *   Beneficio promedio: **$6,777,032.30**
    *   Intervalo de confianza del 95%: [**$1,479,105.53**, **$12,298,424.41**]
    *   Riesgo de pérdidas: **0.50%**

*   **Región 2:**
    *   Beneficio promedio: **$28,807,085.25**
    *   Intervalo de confianza del 95%: [**$23,454,602.71**, **$35,023,917.94**]
    *   Riesgo de pérdidas: **0.00%**

### Análisis y Decisión

Según las condiciones del proyecto, solo se deben mantener las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a este criterio, se debe seleccionar la región con el beneficio promedio más alto.

Todas las regiones (Región 0 con 0.00% de riesgo, Región 1 con 0.50% de riesgo y Región 2 con 0.00% de riesgo) cumplen con el criterio de tener un riesgo de pérdidas inferior al 2.5%.

Al comparar los beneficios promedio de las regiones que cumplen:

*   Región 0: $26,674,574.75
*   Región 1: $6,777,032.30
*   Región 2: $28,807,085.25

La **Región 2** presenta el beneficio promedio más alto entre todas las regiones ($28,807,085.25), además de tener un riesgo de pérdidas del 0.00%, lo cual es ideal. La Región 0 también muestra un buen beneficio y un 0.00% de riesgo, pero su beneficio promedio es ligeramente inferior al de la Región 2. La Región 1, aunque cumple con el umbral de riesgo, tiene un beneficio promedio significativamente menor.

### Propuesta Final

Basado en el análisis de riesgo y beneficio utilizando bootstrapping, la **Región 2** es la más prometedora para el desarrollo de pozos petrolíferos. Ofrece el beneficio promedio más alto y un riesgo de pérdidas del 0.00%, cumpliendo con todos los criterios establecidos.

### Comparación con la Elección Previa (Punto 4.3)

La elección actual **coincide** con la propuesta previa del punto 4.3, donde la Región 2 también fue seleccionada como la candidata más fuerte basándose únicamente en la ganancia potencial calculada para los 200 pozos más prometedores. La incorporación del análisis de bootstrapping y riesgo ha reforzado esta conclusión, confirmando que la Región 2 no solo tiene el mayor potencial de ganancia sino que también lo hace con un riesgo de pérdidas nulo, lo cual es altamente favorable.

## Conclusiones y Propuesta de Región Final (Después de Bootstrapping)

Después de realizar el análisis de riesgos y beneficios utilizando la técnica de bootstrapping con 1000 muestras, los resultados para cada región son los siguientes:

*   **Región 0:**
    *   Beneficio promedio: **$26,674,574.75**
    *   Intervalo de confianza del 95%: [**$21,464,572.89**, **$32,462,222.97**]
    *   Riesgo de pérdidas: **0.00%**

*   **Región 1:**
    *   Beneficio promedio: **$6,777,032.30**
    *   Intervalo de confianza del 95%: [**$1,479,105.53**, **$12,298,424.41**]
    *   Riesgo de pérdidas: **0.50%**

*   **Región 2:**
    *   Beneficio promedio: **$28,807,085.25**
    *   Intervalo de confianza del 95%: [**$23,454,602.71**, **$35,023,917.94**]
    *   Riesgo de pérdidas: **0.00%**

### Análisis y Decisión

Según las condiciones del proyecto, solo se deben mantener las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a este criterio, se debe seleccionar la región con el beneficio promedio más alto.

Todas las regiones (Región 0 con 0.00% de riesgo, Región 1 con 0.50% de riesgo y Región 2 con 0.00% de riesgo) cumplen con el criterio de tener un riesgo de pérdidas inferior al 2.5%.

Al comparar los beneficios promedio de las regiones que cumplen:

*   Región 0: $26,674,574.75
*   Región 1: $6,777,032.30
*   Región 2: $28,807,085.25

La **Región 2** presenta el beneficio promedio más alto entre todas las regiones ($28,807,085.25), además de tener un riesgo de pérdidas del 0.00%, lo cual es ideal. La Región 0 también muestra un buen beneficio y un 0.00% de riesgo, pero su beneficio promedio es ligeramente inferior al de la Región 2. La Región 1, aunque cumple con el umbral de riesgo, tiene un beneficio promedio significativamente menor.

### Propuesta Final

Basado en el análisis de riesgo y beneficio utilizando bootstrapping, la **Región 2** es la más prometedora para el desarrollo de pozos petrolíferos. Ofrece el beneficio promedio más alto y un riesgo de pérdidas del 0.00%, cumpliendo con todos los criterios establecidos.

### Comparación con la Elección Previa (Punto 4.3)

La elección actual **coincide** con la propuesta previa del punto 4.3, donde la Región 2 también fue seleccionada como la candidata más fuerte basándose únicamente en la ganancia potencial calculada para los 200 pozos más prometedores. La incorporación del análisis de bootstrapping y riesgo ha reforzado esta conclusión, confirmando que la Región 2 no solo tiene el mayor potencial de ganancia sino que también lo hace con un riesgo de pérdidas nulo, lo cual es altamente favorable.


## Conclusiones y Propuesta de Región Final (Después de Bootstrapping)

Después de realizar el análisis de riesgos y beneficios utilizando la técnica de bootstrapping con 1000 muestras, los resultados para cada región son los siguientes:

*   **Región 0:**
    *   Beneficio promedio: **$26,674,574.75**
    *   Intervalo de confianza del 95%: [**$21,464,572.89**, **$32,462,222.97**]
    *   Riesgo de pérdidas: **0.00%**

*   **Región 1:**
    *   Beneficio promedio: **$6,777,032.30**
    *   Intervalo de confianza del 95%: [**$1,479,105.53**, **$12,298,424.41**]
    *   Riesgo de pérdidas: **0.50%**

*   **Región 2:**
    *   Beneficio promedio: **$28,807,085.25**
    *   Intervalo de confianza del 95%: [**$23,454,602.71**, **$35,023,917.94**]
    *   Riesgo de pérdidas: **0.00%**

### Análisis y Decisión

Según las condiciones del proyecto, solo se deben mantener las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a este criterio, se debe seleccionar la región con el beneficio promedio más alto.

Todas las regiones (Región 0 con 0.00% de riesgo, Región 1 con 0.50% de riesgo y Región 2 con 0.00% de riesgo) cumplen con el criterio de tener un riesgo de pérdidas inferior al 2.5%.

Al comparar los beneficios promedio de las regiones que cumplen:

*   Región 0: $26,674,574.75
*   Región 1: $6,777,032.30
*   Región 2: $28,807,085.25

La **Región 2** presenta el beneficio promedio más alto entre todas las regiones ($28,807,085.25), además de tener un riesgo de pérdidas del 0.00%, lo cual es ideal. La Región 0 también muestra un buen beneficio y un 0.00% de riesgo, pero su beneficio promedio es ligeramente inferior al de la Región 2. La Región 1, aunque cumple con el umbral de riesgo, tiene un beneficio promedio significativamente menor.

### Propuesta Final

Basado en el análisis de riesgo y beneficio utilizando bootstrapping, la **Región 2** es la más prometedora para el desarrollo de pozos petrolíferos. Ofrece el beneficio promedio más alto y un riesgo de pérdidas del 0.00%, cumpliendo con todos los criterios establecidos.

### Comparación con la Elección Previa (Punto 4.3)

La elección actual **coincide** con la propuesta previa del punto 4.3, donde la Región 2 también fue seleccionada como la candidata más fuerte basándose únicamente en la ganancia potencial calculada para los 200 pozos más prometedores. La incorporación del análisis de bootstrapping y riesgo ha reforzado esta conclusión, confirmando que la Región 2 no solo tiene el mayor potencial de ganancia sino que también lo hace con un riesgo de pérdidas nulo, lo cual es altamente favorable.


## Conclusiones y Propuesta de Región Final (Después de Bootstrapping)

Después de realizar el análisis de riesgos y beneficios utilizando la técnica de bootstrapping con 1000 muestras, los resultados para cada región son los siguientes:

*   **Región 0:**
    *   Beneficio promedio: **$26,674,574.75**
    *   Intervalo de confianza del 95%: [**$21,464,572.89**, **$32,462,222.97**]
    *   Riesgo de pérdidas: **0.00%**

*   **Región 1:**
    *   Beneficio promedio: **$6,777,032.30**
    *   Intervalo de confianza del 95%: [**$1,479,105.53**, **$12,298,424.41**]
    *   Riesgo de pérdidas: **0.50%**

*   **Región 2:**
    *   Beneficio promedio: **$28,807,085.25**
    *   Intervalo de confianza del 95%: [**$23,454,602.71**, **$35,023,917.94**]
    *   Riesgo de pérdidas: **0.00%**

### Análisis y Decisión

Según las condiciones del proyecto, solo se deben mantener las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a este criterio, se debe seleccionar la región con el beneficio promedio más alto.

Todas las regiones (Región 0 con 0.00% de riesgo, Región 1 con 0.50% de riesgo y Región 2 con 0.00% de riesgo) cumplen con el criterio de tener un riesgo de pérdidas inferior al 2.5%.

Al comparar los beneficios promedio de las regiones que cumplen:

*   Región 0: $26,674,574.75
*   Región 1: $6,777,032.30
*   Región 2: $28,807,085.25

La **Región 2** presenta el beneficio promedio más alto entre todas las regiones ($28,807,085.25), además de tener un riesgo de pérdidas del 0.00%, lo cual es ideal. La Región 0 también muestra un buen beneficio y un 0.00% de riesgo, pero su beneficio promedio es ligeramente inferior al de la Región 2. La Región 1, aunque cumple con el umbral de riesgo, tiene un beneficio promedio significativamente menor.

### Propuesta Final

Basado en el análisis de riesgo y beneficio utilizando bootstrapping, la **Región 2** es la más prometedora para el desarrollo de pozos petrolíferos. Ofrece el beneficio promedio más alto y un riesgo de pérdidas del 0.00%, cumpliendo con todos los criterios establecidos.

### Comparación con la Elección Previa (Punto 4.3)

La elección actual **coincide** con la propuesta previa del punto 4.3, donde la Región 2 también fue seleccionada como la candidata más fuerte basándose únicamente en la ganancia potencial calculada para los 200 pozos más prometedores. La incorporación del análisis de bootstrapping y riesgo ha reforzado esta conclusión, confirmando que la Región 2 no solo tiene el mayor potencial de ganancia sino que también lo hace con un riesgo de pérdidas nulo, lo cual es altamente favorable.


## Conclusiones y Propuesta de Región Final (Después de Bootstrapping)

Después de realizar el análisis de riesgos y beneficios utilizando la técnica de bootstrapping con 1000 muestras, los resultados para cada región son los siguientes:

*   **Región 0:**
    *   Beneficio promedio: **$26,674,574.75**
    *   Intervalo de confianza del 95%: [**$21,464,572.89**, **$32,462,222.97**]
    *   Riesgo de pérdidas: **0.00%**

*   **Región 1:**
    *   Beneficio promedio: **$6,777,032.30**
    *   Intervalo de confianza del 95%: [**$1,479,105.53**, **$12,298,424.41**]
    *   Riesgo de pérdidas: **0.50%**

*   **Región 2:**
    *   Beneficio promedio: **$28,807,085.25**
    *   Intervalo de confianza del 95%: [**$23,454,602.71**, **$35,023,917.94**]
    *   Riesgo de pérdidas: **0.00%**

### Análisis y Decisión

Según las condiciones del proyecto, solo se deben mantener las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a este criterio, se debe seleccionar la región con el beneficio promedio más alto.

Todas las regiones (Región 0 con 0.00% de riesgo, Región 1 con 0.50% de riesgo y Región 2 con 0.00% de riesgo) cumplen con el criterio de tener un riesgo de pérdidas inferior al 2.5%.

Al comparar los beneficios promedio de las regiones que cumplen:

*   Región 0: $26,674,574.75
*   Región 1: $6,777,032.30
*   Región 2: $28,807,085.25

La **Región 2** presenta el beneficio promedio más alto entre todas las regiones ($28,807,085.25), además de tener un riesgo de pérdidas del 0.00%, lo cual es ideal. La Región 0 también muestra un buen beneficio y un 0.00% de riesgo, pero su beneficio promedio es ligeramente inferior al de la Región 2. La Región 1, aunque cumple con el umbral de riesgo, tiene un beneficio promedio significativamente menor.

### Propuesta Final

Basado en el análisis de riesgo y beneficio utilizando bootstrapping, la **Región 2** es la más prometedora para el desarrollo de pozos petrolíferos. Ofrece el beneficio promedio más alto y un riesgo de pérdidas del 0.00%, cumpliendo con todos los criterios establecidos.

### Comparación con la Elección Previa (Punto 4.3)

La elección actual **coincide** con la propuesta previa del punto 4.3, donde la Región 2 también fue seleccionada como la candidata más fuerte basándose únicamente en la ganancia potencial calculada para los 200 pozos más prometedores. La incorporación del análisis de bootstrapping y riesgo ha reforzado esta conclusión, confirmando que la Región 2 no solo tiene el mayor potencial de ganancia sino que también lo hace con un riesgo de pérdidas nulo, lo cual es altamente favorable.

## Summary:

### Q&A
Based on the analysis, Region 2 is the most likely to be profitable and is recommended for proceeding with well development. It demonstrated the highest average profit (\$28,807,085.25) and a 0.00% risk of losses, meeting the specified risk threshold of 2.5%. Region 0 is a close second with a profit of \$26,674,574.75 and 0.00% risk, while Region 1 has a significantly lower average profit of \$6,777,032.30.

### Data Analysis Key Findings
*   **Financial Parameters & Breakeven Analysis:**
    *   A budget of \$100,000,000 for 200 wells translates to a cost of \$500,000 per well.
    *   With a revenue of \$4,500 per unit, the breakeven product volume required per well is 111.111 thousand barrels.
    *   The mean predicted reserves for all regions (Region 0: 92.980, Region 1: 68.723, Region 2: 94.884 thousand barrels) are significantly below the breakeven volume, indicating that random well selection would likely result in losses.
*   **Initial Profit Potential (without risk analysis):**
    *   Selecting the top 200 predicted wells, Region 2 showed the highest potential profit at \$69,426,631.59, followed by Region 0 (\$65,048,474.95), and Region 1 (\$24,150,866.97).
*   **Bootstrapping Results (1000 samples, 500 wells sampled, top 200 selected):**
    *   **Region 0:** Average profit of \$26,674,574.75 with a 95% confidence interval of [\$21,464,572.89, \$32,462,222.97] and a 0.00% risk of losses.
    *   **Region 1:** Average profit of \$6,777,032.30 with a 95% confidence interval of [\$1,479,105.53, \$12,298,424.41] and a 0.50% risk of losses.
    *   **Region 2:** Average profit of \$28,807,085.25 with a 95% confidence interval of [\$23,454,602.71, \$35,023,917.94] and a 0.00% risk of losses.
*   **Final Selection:** All regions met the risk threshold of less than 2.5%. Among these, Region 2 presented the highest average profit of \$28,807,085.25 with a 0.00% risk of losses, making it the most favorable option.

### Insights or Next Steps
*   The initial assessment of mean reserves highlighted the critical need for strategic well selection, which was subsequently confirmed by the profitability analysis of the top 200 wells.
*   The robustness of Region 2's profitability is reinforced by the bootstrapping analysis, which showed the highest average profit and zero risk of loss, aligning with the initial higher profit potential observation.
